In [ ]:
from rmprocs.dm import * 
from rmprocs.suavizamiento import * 

import pandas as pd 
import numpy as np 

from matplotlib import pyplot as plt 
import seaborn as sns

In [ ]:
oDmApp, PROJECT_FOLDER = get_oDmApp()

def log(x): 
    x = x + "\n"
    oDmApp.ControlBars.Output.Write(x)

In [ ]:
table = "mb_final_smooth_r3a"

In [ ]:
bm = get_dm_table(table, oDmApp)
bm["VOL"] = bm["XINC"] * bm["YINC"] * bm["ZINC"] 

In [ ]:
def summarize_bands(df: pd.DataFrame, dist: float, level_thickness: float) -> pd.DataFrame:
    """
    Summarize KDTree batching bands by Z levels.

    Parameters
    ----------
    df : DataFrame
        Must contain columns ["XC", "YC", "ZC"].
    dist : float
        Neighborhood radius used for smoothing. Also used as overlap.
    level_thickness : float
        Core band thickness along Z.

    Returns
    -------
    summary : DataFrame
        One row per core band with:
        - band_id: sequential band index starting at 1
        - core_lo, core_hi: Z bounds of the core band [lo, hi)
        - z_center: midpoint of the core band ( (lo+hi)/2 )
        - cx, cy, cz: centroid (mean) of core-band cells (NaN if core empty)
        - n_indexed: number of cells in expanded band (core ± dist)
        - n_smoothed: number of cells in core band
    """
    if level_thickness <= 0:
        raise ValueError("level_thickness must be > 0")
    if "XC" not in df or "YC" not in df or "ZC" not in df:
        raise KeyError('df must contain columns "XC", "YC", and "ZC"')

    # Use float32 views for masks and any potential heavy ops
    z = df["ZC"].to_numpy(dtype=np.float32, copy=False)
    x = df["XC"].to_numpy(dtype=np.float32, copy=False)
    y = df["YC"].to_numpy(dtype=np.float32, copy=False)

    zmin = float(np.nanmin(z))
    zmax = float(np.nanmax(z))
    overlap = float(dist)

    # Build the same non-overlapping core windows you use in smoothing
    level_edges = []
    z_start = zmin
    while z_start <= zmax:
        z_end = z_start + level_thickness
        level_edges.append((z_start, z_end))
        z_start = z_end

    rows = []
    for li, (core_lo, core_hi) in enumerate(level_edges, 1):
        exp_lo = core_lo - overlap
        exp_hi = core_hi + overlap

        core_mask = (z >= core_lo) & (z < core_hi)
        exp_mask  = (z >= exp_lo)  & (z < exp_hi)

        n_smoothed = int(np.count_nonzero(core_mask))
        n_indexed  = int(np.count_nonzero(exp_mask))

        if n_smoothed > 0:
            # Centroid over core cells
            cx = float(np.nanmean(x[core_mask]))
            cy = float(np.nanmean(y[core_mask]))
            cz = float(np.nanmean(z[core_mask]))
        else:
            cx = cy = cz = np.nan

        rows.append({
            "band_id":   li,
            "core_lo":   float(core_lo),
            "core_hi":   float(core_hi),
            "z_center":  float((core_lo + core_hi) * 0.5),
            "cx":        cx,
            "cy":        cy,
            "cz":        cz,
            "n_indexed": n_indexed,
            "n_smoothed": n_smoothed,
        })

    return pd.DataFrame(rows)

In [ ]:
# lt = 5
# res = summarize_bands(bm, dist=10, level_thickness=lt)

In [ ]:
# fig, axs = plt.subplots(1, 2, figsize=(10, 4))
# sns.histplot(bm, x="ZC", binwidth=lt, ax=axs[0])
# sns.lineplot(res, x="z_center", y="n_indexed", ax=axs[0])
# sns.lineplot(res, x="z_center", y="n_smoothed", ax=axs[0])

In [ ]:
# Suppose you have summarize_bands_xyz (or swap in summarize_bands_xyz_lean)


size

print("Chosen cubic tile size:", size)
print("Diagnostics:", stats)

# Then call your stable smoother with x_size=y_size=z_size=size
# suavizar_batched_xyz_multi_stable(bm_df, col, dists, out_cols, size, size, size, ...)


In [ ]:
col = "CATE"
dists = [10, 20]

out_col = f"{col}_SUAV"
out_cols = [f"{out_col}_{d}" for d in dists]

suavizar_batched_xyz_multi_stable(
    bm, col, dists, out_cols, 
    x_size=50, y_size=50, z_size=50, 
)

In [ ]:
report = report_volume_variation(bm, "VOL", col, "S10")
report